In [1]:
# selenium-related
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException    
from selenium.webdriver.common.by import By     
from selenium.common import exceptions
from selenium.webdriver.support.select import Select as WebDriverSelect
from selenium.common.exceptions import ElementNotVisibleException

# other necessary ones
#from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import time
import re
import datetime

## Scraping POST's liked Users 

In [ ]:
# set options as you wish
option = Options()
option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")

with open('facebook_credentials.txt') as file:
    EMAIL = file.readline().split('"')[1]
    PASSWORD = file.readline().split('"')[1]
    
driver = webdriver.Chrome(executable_path="./chromedriver", options=option)
driver.get("http://facebook.com")
driver.maximize_window()
wait = WebDriverWait(driver, 30)
email_field = wait.until(EC.visibility_of_element_located((By.NAME, 'email')))
email_field.send_keys(EMAIL)
pass_field = wait.until(EC.visibility_of_element_located((By.NAME, 'pass')))
pass_field.send_keys(PASSWORD)
pass_field.send_keys(Keys.RETURN)

time.sleep(5)
driver.get('https://m.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier=2026865604156673&av=100078295571647&ext=1645445756&hash=AeT78E_92OYHRDTZ9zY') # once logged in, free to open up any target page
 # once logged in, free to open up any target page
    
count=20
while count>1:
    try:
        button=driver.find_element_by_xpath('//*[@id="reaction_profile_pager"]/a/div/div/div/strong')
        button.click()
        count-=1
        time.sleep(2)
    except NoSuchElementException :
        break

time.sleep(5)

with open('user_link.csv', 'w') as file:
    file.write('NPP_liked_user_link \n')
    
for i in driver.find_elements_by_xpath("//div[@class='ib cc _1aj4']"):
        #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);",i)
    time.sleep(2)
    # user account link
    a_element = i.find_element_by_tag_name("a")
    link = a_element.get_attribute('href')
    with open('user_link.csv', 'a') as file:
        for i in range(1):
            file.write(link + '\n')
    #profile_link.append(link)
        #print(profile_link)
driver.close()

df = pd.read_csv('user_link.csv') 
df

## Conrad_K_sharma_Scraping POST liked Users data

In [ ]:
option = Options()
option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")
driver = webdriver.Chrome(executable_path="./chromedriver", options=option)

with open('facebook_credentials.txt') as file:
    EMAIL = file.readline().split('"')[1]
    PASSWORD = file.readline().split('"')[1]
    
driver.get("http://facebook.com")
driver.maximize_window()
wait = WebDriverWait(driver, 30)
email_field = wait.until(EC.visibility_of_element_located((By.NAME, 'email')))
email_field.send_keys(EMAIL)
pass_field = wait.until(EC.visibility_of_element_located((By.NAME, 'pass')))
pass_field.send_keys(PASSWORD)
pass_field.send_keys(Keys.RETURN)

time.sleep(5)

with open('example1.csv', 'w', encoding="utf-8") as file:
    file.write('source_name,link,user_name,follower,infos \n')
    
source_name = 'Conrad K Sangma'
df = pd.read_csv('user_link.csv') # can also index sheet by name or fetch all sheets
mylist = df['user_link'].tolist()

# Using for loop 
for i in mylist: 
    driver.get(i)
    time.sleep(5)
    
# user profile link
    link = driver.current_url 
    # user name
    try:
        user_name = driver.find_element_by_xpath("//div[@class='bi6gxh9e aov4n071']").text
    except NoSuchElementException:
        pass

    # follower count (working)
    try:
        follower = driver.find_element_by_css_selector("span[class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em mdeji52x a5q79mjw g1cxx5fr b1v8xokw m9osqain']").text
    except NoSuchElementException:
        pass
    time.sleep(3)
    try:
        info = driver.find_elements_by_xpath("//div[@class='rq0escxv l9j0dhe7 du4w35lb j83agx80 pfnyh3mw i1fnvgqd gs1a9yip owycx6da btwxx1t3 hv4rvrfc dati1w0a discj3wi b5q2rw42 lq239pai mysgfdmx hddg9phg']")
        for i in info:
            infos = i.text
            infos1 = infos.replace("\n", " ")
            infos2 = infos1.replace(",", " ")
            #print(infos2)    
    except NoSuchElementException:
        pass
    
    #source_name = 'Conrad K Sangma'
    time.sleep(5)
        
    #writing to file
    with open('example1.csv', 'a', encoding="utf-8") as file:
        for i in range(1):
            file.write(source_name + "," + link + "," + user_name + "," + follower + "," + infos2 + '\n')
    #file.close()
    
    driver.back()
driver.close()

## Scraping Friends of friends Data

In [2]:
# set options as you wish
option = Options()
option.add_argument("--disable-infobars")
#option.add_argument("headless")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")
PROXY = "11.456.448.110:8080" # IP:PORT or HOST:PORT(202.136.165.22)
chrome_options = webdriver.ChromeOptions() 
option.add_argument('--proxy-server=%s' % PROXY) 
chrome_options.add_argument('--proxy-server=%s' % PROXY) 
#chrome = webdriver.Chrome(options=chrome_options) 
driver = webdriver.Chrome(executable_path="./chromedriver", options=option)

with open('facebook_credentials.txt') as file:
    EMAIL = file.readline().split('"')[1]
    PASSWORD = file.readline().split('"')[1]
    
driver.get("http://facebook.com")
driver.maximize_window()
wait = WebDriverWait(driver, 30)
email_field = wait.until(EC.visibility_of_element_located((By.NAME, 'email')))
email_field.send_keys(EMAIL)
pass_field = wait.until(EC.visibility_of_element_located((By.NAME, 'pass')))
pass_field.send_keys(PASSWORD)
pass_field.send_keys(Keys.RETURN)

time.sleep(5)

df = pd.read_csv('Conrad_k_Sangma.csv') # can also index sheet by name or fetch all sheets
mylist = df['link'].tolist()

name = []
url = []
s_name = []
follower1 = []
info = []


# Using for loop 
for i in mylist:
    driver.get(i)
    time.sleep(4)

    try:
        WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Friends']"))).click()
    except:
        pass
    time.sleep(2)
    
    SCROLL_PAUSE_TIME = 0.5
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    driver.execute_script("window.scrollTo(0, -document.body.scrollTop);")
    time.sleep(3)
    
    elements = driver.find_elements_by_xpath("//div[@class='buofh1pr hv4rvrfc']")

    for i in range(len(elements)):
        time.sleep(4)
        try:
            source_name = driver.find_element_by_xpath("//div[@class='j83agx80 cbu4d94t obtkqiv7 sv5sfqaa']").text
            s_name.append(source_name)
        except:
            s_name.append('N/A')   
        time.sleep(4)
        
        try:
            element = driver.find_elements_by_css_selector("span[class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em mdeji52x a5q79mjw g1cxx5fr lrazzd5p oo9gr5id']")
            element[i].click()
        except:
            pass
        time.sleep(3)
        
        try:
            link = driver.current_url
            url.append(link)
        except:
            url.append('N/A')
        time.sleep(3)

        # user name
        try:
            user_name = driver.find_element_by_xpath("//div[@class='bi6gxh9e aov4n071']").text
            name.append(user_name)
        except:
            pass
        time.sleep(3)

        try:
            follower = driver.find_element_by_css_selector("span[class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em mdeji52x a5q79mjw g1cxx5fr b1v8xokw m9osqain']").text
            follower1.append(follower)
        except NoSuchElementException:
            follower1.append('N/A')
        time.sleep(3)

        try:
            infos = driver.find_elements_by_xpath("//div[@class='sej5wr8e']")
            for i in info:
                #user_info.append(i.text)
                #intro = i.find_element_by_css_selector(".sjgh65i0").text
                infos = i.text
                infos1 = infos.replace("\n", " ")
                infos2 = infos1.replace(",", " ")
                info.append(infos2)
        except:
            info.append('N/A')

        time.sleep(3)
        driver.back()
driver.close

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_2840/973018314.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="./chromedriver", options=option)
C:\Users\ADMIN\AppData\Local\Temp/ipykernel_2840/973018314.py:68: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  elements = driver.find_elements_by_xpath("//div[@class='buofh1pr hv4rvrfc']")
C:\Users\ADMIN\AppData\Local\Temp/ipykernel_2840/973018314.py:75: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  source_name = driver.find_element_by_xpath("//div[@class='j83agx80 cbu4d94t obtkqiv7 sv5sfqaa']").text
C:\Users\ADMIN\AppData\Local\Temp/ipykernel_2840/973018314.py:80: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  element = dr

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='j83agx80 cbu4d94t obtkqiv7 sv5sfqaa']"}
  (Session info: chrome=99.0.4844.51)
Stacktrace:
Backtrace:
	Ordinal0 [0x00667AC3+2587331]
	Ordinal0 [0x005FADD1+2141649]
	Ordinal0 [0x004F3BB8+1063864]
	Ordinal0 [0x005201CE+1245646]
	Ordinal0 [0x005203CB+1246155]
	Ordinal0 [0x0054A612+1418770]
	Ordinal0 [0x005386D4+1345236]
	Ordinal0 [0x00548A0A+1411594]
	Ordinal0 [0x005384A6+1344678]
	Ordinal0 [0x005153F6+1201142]
	Ordinal0 [0x005162E6+1204966]
	GetHandleVerifier [0x0080DF22+1680738]
	GetHandleVerifier [0x008C0DBC+2413564]
	GetHandleVerifier [0x006FD151+563089]
	GetHandleVerifier [0x006FBF13+558419]
	Ordinal0 [0x0060081E+2164766]
	Ordinal0 [0x00605508+2184456]
	Ordinal0 [0x00605650+2184784]
	Ordinal0 [0x0060F5BC+2225596]
	BaseThreadInitThunk [0x75936739+25]
	RtlGetFullPathName_UEx [0x77058AFF+1215]
	RtlGetFullPathName_UEx [0x77058ACD+1165]


In [3]:
a = { "s_name":s_name, 'name': name, 'follower_count': follower1, 'URL': url, 'info':info}
df = pd.DataFrame.from_dict(a, orient='index')
df1=df.transpose()

In [5]:
df1.to_excel('output.xls', index=True)
df1.head()

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_2840/1280492255.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df1.to_excel('output.xls', index=True)


,s_name,name,follower_count,URL,info
0,Prince Elokho,S Adao Khrasi,526 Friends,https://www.facebook.com/profile.php?id=100076...,None
1,Prince Elokho,Ayuo Pao,N/A,https://www.facebook.com/ayuo.ashe,None
2,Prince Elokho,Hriiyia Mao,N/A,https://www.facebook.com/hr.yia,None
3,Prince Elokho,Anio Khudai Brightson,3.1K Friends,https://www.facebook.com/khudai.v.born,None
4,Prince Elokho,Ajihrii Aruo Kade,479 Friends,https://www.facebook.com/profile.php?id=100078...,None


In [6]:
df1.shape

(14, 5)